In [7]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=12

println("Number of variables: n=",n)

@polyvar x[1:n]# variables

#function to get a random quadratic polynomial of variables x(T)
function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# ball constraints on subsets of variables
u=4# clique size
p=floor(Int64,n/u) #number of cliques

# indices of variables on each clique
I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u 
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p] #vector of separable polynomials on each clique
f=sum(vecf)

# ball constraints on each clique
m=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,m/p)
J=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(J,[(p-1)*r+1:m])

g=Vector{Polynomial{true,Float64}}(undef,m)


# get a random point satisfies the inequality constraints
randx=rand(Float64,n)

for j in 1:p
    randx[I[j]]=randx[I[j]]./sum(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in J[j]
        g[i]=generate_random_poly(I[j])
        g[i]+=0.125-g[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
    g[J[j][1]]=1-sum(x[I[j]])
end

m=length(g)
println("Number of inequality constraints: m=",m)


l=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variables: n=12
Number of inequality constraints: m=3
Number of equality constraints: l=3


In [8]:
# for j in 1:p
#     g=[g;(1-sum(x[I[j]]))^2]
# end

In [11]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

k=3
t=2
s=4

@time opt_val1,opt_sol=InterRelax.RelaxSparseCSTS(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,s,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Term sparsity order: t=2
Upper bound on maximal matrix size: s=4
------------------------------------------------------
The clique sizes of varibles:
[5, 4]
[2, 1]
------------------------------------------------------
  Number of cliques: p=

3
  Largest clique size: u=5
  Even symmetry: true
  Maximal matrix size:4
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 139             
  Cones                  : 0               
  Scalar variables       : 108             
  Matrix variables       : 54              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense 

In [10]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl")

k=3
s=4

@time opt_val1,opt_sol=InterRelax.RelaxSparse_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,s,k,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Sparsity order: s=4
------------------------------------------------------
The clique sizes of varibles:
[5, 4]
[2, 1]
------------------------------------------------------


  Number of cliques: p=3
  Largest clique size: u=5
 Maximal matrix size:4
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 139             
  Cones                  : 0               
  Scalar variables       : 108             
  Matrix variables       : 54              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense 

In [57]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl");

k=1
s=28
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=1
Sparsity order: s=28
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[26, 16, 15]
[1, 4, 1]
------------------------------------------------------
  Number of cliques: p=6
  Largest clique size: u=26
 Maximal matrix size:27
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 9457            
  Cones                  : 0               
  Scalar variables       : 9010            
  Matrix variables       : 117             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 10
Eliminator terminated.
Eliminator - tries                  : 1              

In [28]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl");

k=2

opt=InterRelax.TSSOS_CS(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,k);

**Semidefinite relaxation based on Putinar's Positivstellensatz**
Relaxation order: k=2
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[26, 16, 15]
[1, 4, 1]
------------------------------------------------------
Obtained the variable cliques in 0.002352839 seconds. The maximal size of cliques is 26.
Assembling the SDP...
There are 50636 affine constraints.
SDP assembling time: 2.666728798 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 50636           
  Cones                  : 0               
  Scalar variables       : 4135            
  Matrix variables       : 126             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear depende

In [44]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=2
vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[10]
[1]
------------------------------------------------------
Assembling the SDP...
There are 1001 affine constraints.
SDP assembling time: 0.008555836 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1001            
  Cones                  : 0               
  Scalar variables       : 133             
  Matrix variables       : 13              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00    